In [1]:
from pgtools import gff_parser
import os
import pandas as pd

In [5]:
gffs_dir = "/home/pampuch/studia/magisterka/final_summaries/gffs/"
panaroo_cores_dir = "/home/pampuch/studia/magisterka/final_summaries/core_summaries_new/panaroo_model/"

In [3]:
for dir in os.listdir(panaroo_cores_dir):
    print(dir)
    for f in os.listdir(os.path.join(panaroo_cores_dir,dir)):
        print(f)

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented
panaroo_core_summary.csv
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated
panaroo_core_summary.csv
klebsiella_subset
panaroo_core_summary.csv
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1
panaroo_core_summary.csv
GPSC_subset
panaroo_core_summary.csv


In [241]:
def summarise_panaroo_core(core_res_dir):
    res_dict = {model: {"core": None, "noncore":None} for model in os.listdir(core_res_dir) if not model.endswith("parsed")}
    for dir in os.listdir(core_res_dir):
        csv_name = "panaroo_core_summary.csv"
        cds_col = "CDS"

        df = pd.read_csv(os.path.join(core_res_dir, dir,csv_name)).dropna()
        print(df.columns)
        # print(len(df))

        # print(os.path.join(core_res_dir, dir, dataset, csv_name))
        print(len(df))
        # if dir.startswith("panaroo"):
        df = df[df["CDS"].apply(lambda x: x.split("_")[1] != "refound")]
        df["seq_ann"] = df["seq name"] + "." + df["CDS"]
        res_dict[dir]["core"] = set(df[df["core status"] == True]["seq_ann"].unique())
        res_dict[dir]["noncore"] = set(df[df["core status"] == False]["seq_ann"].unique())
    return res_dict
panaroo_dict = summarise_panaroo_core(panaroo_cores_dir)

Index(['cluster name', 'cluster size', 'core status', 'seq name', 'CDS'], dtype='object')
311100
Index(['cluster name', 'cluster size', 'core status', 'seq name', 'CDS'], dtype='object')
308948
Index(['cluster name', 'cluster size', 'core status', 'seq name', 'CDS'], dtype='object')
106900
Index(['cluster name', 'cluster size', 'core status', 'seq name', 'CDS'], dtype='object')
308976
Index(['cluster name', 'cluster size', 'core status', 'seq name', 'CDS'], dtype='object')
45588


### Co zrobić z gff
1. Jedno zrób bez thresholdowania, drugie z (50%)

In [242]:
# klebsiella

dataset = "klebsiella_subset"

gff_cact = os.path.join(gffs_dir, "cds_inter_cact", dataset, "cds_inter_cact.gff")
gff_cact_core = os.path.join(gffs_dir, "cds_inter_core", dataset, "cds_inter_core.gff")
all_cds = os.path.join(gffs_dir, "cds_gffs", dataset, "all.gff")


In [243]:
panaroo_dict[dataset]

{'core': {'5299_1_3.contig00010.OBOFFKJP_00882',
  '5235_1_4.contig00168.MGBABOJF_02482',
  '5235_1_4.contig00344.MGBABOJF_03709',
  '5150_2_2.contig00013.ILAAFBJJ_02805',
  '5193_8_2.contig00001.MLKBEPLK_00254',
  '5151_6_6.contig00014.NJAAJKJA_02779',
  '5235_6_6.contig00374.LNOJILEH_04468',
  '5151_2_6.contig00004.NALFBGJE_00912',
  '5299_7_4.contig00006.MOOPCBGB_00510',
  '5299_7_4.contig00037.MOOPCBGB_02200',
  '5197_2_1.contig00052.PDEDMGPD_04957',
  '5197_2_1.contig00027.PDEDMGPD_03741',
  '5150_2_2.contig00006.ILAAFBJJ_01586',
  '5235_5_4.contig00053.LLOJMOCI_01544',
  '5193_1_5.contig00033.AHIKKBJF_03662',
  '5151_2_6.contig00006.NALFBGJE_01208',
  '5150_1_3.contig00029.AOMCCKAJ_04295',
  '5235_2_1.contig00111.IJODNBMD_02584',
  '5151_6_6.contig00033.NJAAJKJA_04402',
  '5299_7_4.contig00060.MOOPCBGB_03004',
  '5235_5_12.contig00239.KKAGLBML_03954',
  '5235_6_6.contig00224.LNOJILEH_03467',
  '5150_2_2.contig00036.ILAAFBJJ_04934',
  '5197_2_1.contig00005.PDEDMGPD_01134',
  '5151

In [244]:
gff_cds = gff_parser.parse_joined_gff(all_cds)

In [245]:
print(gff_cds[:3])

[<pgtools.gff_parser.GFF_record object at 0x7fa7cbb12500>, <pgtools.gff_parser.GFF_record object at 0x7fa7cbb10340>, <pgtools.gff_parser.GFF_record object at 0x7fa7cbb111e0>]


In [246]:
ids = [rec.annotation_id for rec in gff_cds]

In [247]:
all_cds_lens = {}
for cds in gff_cds:
    if cds.seq_name in all_cds_lens:
        all_cds_lens[cds.seq_name][cds.annotation_id] = cds.end - cds.start + 1
    else:
        all_cds_lens[cds.seq_name] = {cds.annotation_id : cds.end - cds.start + 1}

In [248]:
len(all_cds_lens)

8281

In [249]:
def get_cds_sum_lens(gff):
    cds_lens = {}
    gff = gff_parser.parse_joined_gff(gff)
    for cds in gff:
        cds_len = cds.end - cds.start + 1
        if cds.seq_name in cds_lens:
            if cds.annotation_id in cds_lens[cds.seq_name]:
                cds_lens[cds.seq_name][cds.annotation_id] += cds_len
            else:
                cds_lens[cds.seq_name][cds.annotation_id] = cds_len
        else:
            cds_lens[cds.seq_name] = {cds.annotation_id : cds_len}
    return cds_lens

In [250]:
core_lens = get_cds_sum_lens(gff_cact_core)
cact_lens = get_cds_sum_lens(gff_cact)

In [251]:
len(core_lens)

42

In [252]:
def dict_to_ann_set(mapped_lens):
    res_cds = []
    for seq_name, mapped in mapped_lens.items():
        for ann_id, ann_len in mapped.items():
            res_cds.append(f"{seq_name}.{ann_id}")
    return set(res_cds)  
    
def filter_cds_lens(cds_lens, mapped_lens, thr=0.5):
    res_cds = []
    for seq_name in mapped_lens:
        full_lens = cds_lens[seq_name]
        mapped = mapped_lens[seq_name]
        for ann_id, ann_len in mapped.items():
            if ann_len >= 0.3 * full_lens[ann_id]:
                res_cds.append(f"{seq_name}.{ann_id}")
    return set(res_cds)
    
signif_core = filter_cds_lens(all_cds_lens, core_lens)
signif_cact = filter_cds_lens(all_cds_lens, cact_lens)
all_core = dict_to_ann_set(core_lens)
all_cact = dict_to_ann_set(cact_lens)
source_all = dict_to_ann_set(all_cds_lens)

In [253]:
klebsiella_summary = {"panaroo": {"core" : panaroo_dict[dataset]["core"], "all": panaroo_dict[dataset]["core"].union(panaroo_dict[dataset]["noncore"])}, "cactus": {"core": all_core, "all": all_cact}, "cactus significant": {"core": signif_core, "all": signif_cact}, "source data":{"all": source_all}}

In [254]:
for model in klebsiella_summary:
    print(model)
    for category, annots in klebsiella_summary[model].items():
        print(category, len(annots))

panaroo
core 79826
all 106471
cactus
core 1000
all 25469
cactus significant
core 1000
all 24971
source data
all 112337


In [11]:
def get_cds_sum_lens(gff):
    cds_lens = {}
    gff = gff_parser.parse_joined_gff(gff)
    for cds in gff:
        cds_len = cds.end - cds.start + 1
        if cds.seq_name in cds_lens:
            if cds.annotation_id in cds_lens[cds.seq_name]:
                cds_lens[cds.seq_name][cds.annotation_id] += cds_len
            else:
                cds_lens[cds.seq_name][cds.annotation_id] = cds_len
        else:
            cds_lens[cds.seq_name] = {cds.annotation_id : cds_len}
    return cds_lens

def dict_to_ann_set(mapped_lens):
    res_cds = []
    for seq_name, mapped in mapped_lens.items():
        for ann_id, ann_len in mapped.items():
            res_cds.append(f"{seq_name}.{ann_id}")
    return set(res_cds)  
    
def filter_cds_lens(cds_lens, mapped_lens, thr=0.5):
    res_cds = []
    for seq_name in mapped_lens:
        full_lens = cds_lens[seq_name]
        mapped = mapped_lens[seq_name]
        for ann_id, ann_len in mapped.items():
            if ann_len >= 0.3 * full_lens[ann_id]:
                res_cds.append(f"{seq_name}.{ann_id}")
    return set(res_cds)

def summarise_dataset(dataset):
    gff_cact = os.path.join(gffs_dir, "cds_inter_cact", dataset, "cds_inter_cact.gff")
    gff_cact_core = os.path.join(gffs_dir, "cds_inter_core", dataset, "cds_inter_core.gff")
    all_cds = os.path.join(gffs_dir, "cds_gffs", dataset, "all.gff") 

    gff_cds = gff_parser.parse_joined_gff(all_cds)
    all_cds_lens = {}
    for cds in gff_cds:
        if cds.seq_name in all_cds_lens:
            all_cds_lens[cds.seq_name][cds.annotation_id] = cds.end - cds.start + 1
        else:
            all_cds_lens[cds.seq_name] = {cds.annotation_id : cds.end - cds.start + 1}   
    core_lens = get_cds_sum_lens(gff_cact_core)
    cact_lens = get_cds_sum_lens(gff_cact)
    
    signif_core = filter_cds_lens(all_cds_lens, core_lens)
    signif_cact = filter_cds_lens(all_cds_lens, cact_lens)
    all_core = dict_to_ann_set(core_lens)
    all_cact = dict_to_ann_set(cact_lens)
    source_all = dict_to_ann_set(all_cds_lens)

    return {dataset: {"panaroo": {"core" : panaroo_dict[dataset]["core"], "all": panaroo_dict[dataset]["core"].union(panaroo_dict[dataset]["noncore"])}, "cactus": {"core": all_core, "all": all_cact}, "cactus significant": {"core": signif_core, "all": signif_cact}, "source data":{"all": source_all}}}

# Get CDS coverage for cactus GPSC

In [13]:
dataset="GPSC_subset"
gpsc_gff = os.path.join(gffs_dir, "cds_inter_cact", dataset, "cds_inter_cact.gff")

In [15]:
lens_gpsc = get_cds_sum_lens(gpsc_gff)

In [18]:
sum_gpsc = 0

for k, vals in lens_gpsc.items():
    for annot, ann_len in vals.items():
        sum_gpsc += ann_len

In [19]:
sum_gpsc

14121688

In [21]:
non_cds_sum= 16448652.0

In [22]:
sum_gpsc/non_cds_sum

0.8585316292180052

In [256]:
all_sumamries = {}
for dataset in os.listdir(panaroo_cores_dir):
    print(dataset)
    all_sumamries.update(summarise_dataset(dataset))

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated
klebsiella_subset
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1
GPSC_subset


In [257]:
for dataset in all_sumamries:
    print(dataset)
    for model in all_sumamries[dataset]:
        print(model)
        for category, annots in all_sumamries[dataset][model].items():
            print(category, len(annots))

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented
panaroo
core 298741
all 306923
cactus
core 19775
all 135931
cactus significant
core 19137
all 131856
source data
all 317753
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated
panaroo
core 300758
all 308426
cactus
core 267295
all 293441
cactus significant
core 265417
all 291517
source data
all 319160
klebsiella_subset
panaroo
core 79826
all 106471
cactus
core 1000
all 25469
cactus significant
core 1000
all 24971
source data
all 112337
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1
panaroo
core 301383
all 308678
cactus
core 286750
all 297984
cactus significant
core 284853
all 295928
source data
all 316855
GPSC_subset
panaroo
core 34263
all 45339
cactus
core 266
all 17223
cactus significant
core 259
all 16905
source data
all 47656


In [258]:
all_sumamries["GPSC_subset"]["panaroo"]["all"] = set([i.replace(".velvet.", ".") for i in all_sumamries["GPSC_subset"]["panaroo"]["all"]])

In [259]:
all_sumamries["GPSC_subset"]["panaroo"]["core"] = set([i.replace(".velvet.", ".") for i in all_sumamries["GPSC_subset"]["panaroo"]["core"]])

## Podsumowanie

Dla każdego datasetu taka tabelka że:
1. ile wszystkich cds ogólnie
2. ile cds w panaroo
3. (ile cds w uzupełnieniu panaroo)
4. ile cds w cactusie ogólnie
5. ile cds w cactusie ogólnie z pokryciem sumarycznym 50%
6. ile cds w cactusie core
7. ile cds w cactusie core z pokryciem sumrycznym 50 %
8. ile cds i w panaroo i w cactusie
9. ile cds i w panaroo core i w cactusie core
->  Jeśli będziesz miała też dla uzupełnienia panaroo - może rozdziel na dwie tabele (ale dodawanie tego jest chyba na razie bez sensu, jeśli będzie trzeba - po prostu usuń te, co mają dlugość 1)

Ponieważ wyniki są dość podobne, zrób tabelę tylko dla tych, co mają thr 50%

In [260]:
def datasets_mapping(x):
    if x.startswith("sim"):
        x= x.replace("sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1", "E. coli ")
    else:
        x= x.replace("_subset", "")
        x= x.replace("GPSC", "Pneumococcus")
        x = x.replace("klebsiella", "Klebsiella")
    return x.strip().replace("_","")


In [261]:
ann_n = {}
df_cols = []
df_col_names = ["Dataset", "Model", "Liczba wszystkich adnotacji", "Liczba adnotacji w genomie rdzeniowym"]
for dataset, models in all_sumamries.items():
    ann_n[dataset] = {}
    for model, core_status in models.items():
        if model == "source data":
            continue
        ann_n[dataset][model] = {}
        ann_all = len(core_status["all"])
        
        ann_n[dataset][model]["Liczba wszystkich adnotacji"] = ann_all
        ann_n[dataset][model]["Liczba adnotacji w genomie rdzeniowym"] = len(core_status["core"])
        df_cols.append([dataset, model,ann_all, len(core_status["core"])])

print(ann_n)

{'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented': {'panaroo': {'Liczba wszystkich adnotacji': 306923, 'Liczba adnotacji w genomie rdzeniowym': 298741}, 'cactus': {'Liczba wszystkich adnotacji': 135931, 'Liczba adnotacji w genomie rdzeniowym': 19775}, 'cactus significant': {'Liczba wszystkich adnotacji': 131856, 'Liczba adnotacji w genomie rdzeniowym': 19137}}, 'sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated': {'panaroo': {'Liczba wszystkich adnotacji': 308426, 'Liczba adnotacji w genomie rdzeniowym': 300758}, 'cactus': {'Liczba wszystkich adnotacji': 293441, 'Liczba adnotacji w genomie rdzeniowym': 267295}, 'cactus significant': {'Liczba wszystkich adnotacji': 291517, 'Liczba adnotacji w genomie rdzeniowym': 265417}}, 'klebsiella_subset': {'panaroo': {'Liczba wszystkich adnotacji': 106471, 'Liczba adnotacji w genomie rdzeniowym': 79826}, 'cactus': {'Liczba wszystkich adnotacji': 25469, 'Liczba adnotacji w genomie rdzeniowym': 1000}, 'cactus significant': {'Liczba wszystkich adnotacj

In [262]:
df_basic = pd.DataFrame(columns=df_col_names, data=df_cols)
df_basic["Dataset"] = df_basic["Dataset"].apply(datasets_mapping)
df_basic["Model"] = df_basic["Model"].apply(lambda x: "cactus (powyżej progu pokrycia)" if x =="cactus significant" else x)

In [263]:
df_basic 

,Dataset,Model,Liczba wszystkich adnotacji,Liczba adnotacji w genomie rdzeniowym
0,E. coli fragmented,panaroo,306923,298741
1,E. coli fragmented,cactus,135931,19775
2,E. coli fragmented,cactus (powyżej progu pokrycia),131856,19137
3,E. coli contaminated,panaroo,308426,300758
4,E. coli contaminated,cactus,293441,267295
5,E. coli contaminated,cactus (powyżej progu pokrycia),291517,265417
6,Klebsiella,panaroo,106471,79826
7,Klebsiella,cactus,25469,1000
8,Klebsiella,cactus (powyżej progu pokrycia),24971,1000
9,E. coli,panaroo,308678,301383


In [264]:
df_basic.to_csv("/home/pampuch/studia/magisterka/final_summaries/bedtools_core/stats_basic.csv", index=False)

In [265]:
stats = []
stats_col_names = ["Dataset", "Narzędzie", "Czułość", "Precyzja"]
for dataset, models in all_sumamries.items():
    truth = all_sumamries[dataset]["panaroo"]["core"]
    # print(len(truth))
    for model, core_status in models.items():
        if model == "panaroo" or model == "source data":
            continue
        inter_ann = truth.intersection(core_status["core"])
        model_n = len(core_status["core"])
        recall = len(inter_ann) / len(truth)
        prec = len(inter_ann) / model_n
        stats.append([dataset, model, recall, prec])

In [266]:
df_stats = pd.DataFrame(columns = stats_col_names, data = stats)
df_stats

,Dataset,Narzędzie,Czułość,Precyzja
0,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented,cactus,0.061856,0.934463
1,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented,cactus significant,0.059985,0.936406
2,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated,cactus,0.857354,0.964687
3,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated,cactus significant,0.856699,0.970771
4,klebsiella_subset,cactus,0.011538,0.921000
5,klebsiella_subset,cactus significant,0.011538,0.921000
6,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1,cactus,0.915768,0.962500
7,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1,cactus significant,0.915689,0.968826
8,GPSC_subset,cactus,0.006625,0.853383
9,GPSC_subset,cactus significant,0.006625,0.876448


In [267]:
df_stats["Dataset"] = df_stats["Dataset"].apply(datasets_mapping)
df_stats["Narzędzie"] = df_stats["Narzędzie"].apply(lambda x: x.split("_")[0])
df_stats["Czułość"] = df_stats["Czułość"].round(3)
df_stats["Precyzja"] = df_stats["Precyzja"].round(3)
df_stats.rename(columns = {"Narzędzie" : "Model"} ,inplace=True)
df_stats["Model"] = df_stats["Model"].apply(lambda x: "cactus (powyżej progu pokrycia)" if x =="cactus significant" else x)
df_stats = df_stats.sort_values(by=["Dataset", "Model"])

In [268]:
df_stats

,Dataset,Model,Czułość,Precyzja
6,E. coli,cactus,0.916,0.963
7,E. coli,cactus (powyżej progu pokrycia),0.916,0.969
2,E. coli contaminated,cactus,0.857,0.965
3,E. coli contaminated,cactus (powyżej progu pokrycia),0.857,0.971
0,E. coli fragmented,cactus,0.062,0.934
1,E. coli fragmented,cactus (powyżej progu pokrycia),0.060,0.936
4,Klebsiella,cactus,0.012,0.921
5,Klebsiella,cactus (powyżej progu pokrycia),0.012,0.921
8,Pneumococcus,cactus,0.007,0.853
9,Pneumococcus,cactus (powyżej progu pokrycia),0.007,0.876


In [269]:
df_stats.to_csv("/home/pampuch/studia/magisterka/final_summaries/bedtools_core/stats_core.csv", index=False)

In [270]:
stats = []
stats_col_names = ["Dataset", "Narzędzie", "Czułość", "Precyzja"]
for dataset, models in all_sumamries.items():
    truth = all_sumamries[dataset]["panaroo"]["all"]
    # print(len(truth))
    for model, core_status in models.items():
        if model == "panaroo" or model == "source data":
            continue
        inter_ann = truth.intersection(core_status["all"])
        model_n = len(core_status["all"])
        recall = len(inter_ann) / len(truth)
        prec = len(inter_ann) / model_n
        stats.append([dataset, model, recall, prec])

In [271]:
df_stats = pd.DataFrame(columns = stats_col_names, data = stats)
df_stats

,Dataset,Narzędzie,Czułość,Precyzja
0,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented,cactus,0.428583,0.967712
1,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented,cactus significant,0.415550,0.967282
2,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated,cactus,0.928129,0.975525
3,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated,cactus significant,0.922017,0.975497
4,klebsiella_subset,cactus,0.228851,0.956692
5,klebsiella_subset,cactus significant,0.224333,0.956510
6,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1,cactus,0.941146,0.974921
7,sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1,cactus significant,0.934644,0.974913
8,GPSC_subset,cactus,0.363771,0.957615
9,GPSC_subset,cactus significant,0.356977,0.957409


In [272]:
df_stats["Dataset"] = df_stats["Dataset"].apply(datasets_mapping)
df_stats["Narzędzie"] = df_stats["Narzędzie"].apply(lambda x: x.split("_")[0])
df_stats["Czułość"] = df_stats["Czułość"].round(3)
df_stats["Precyzja"] = df_stats["Precyzja"].round(3)
df_stats.rename(columns = {"Narzędzie" : "Model"} ,inplace=True)
df_stats["Model"] = df_stats["Model"].apply(lambda x: "cactus (powyżej progu pokrycia)" if x =="cactus significant" else x)
df_stats = df_stats.sort_values(by=["Dataset", "Model"])

In [273]:
df_stats

,Dataset,Model,Czułość,Precyzja
6,E. coli,cactus,0.941,0.975
7,E. coli,cactus (powyżej progu pokrycia),0.935,0.975
2,E. coli contaminated,cactus,0.928,0.976
3,E. coli contaminated,cactus (powyżej progu pokrycia),0.922,0.975
0,E. coli fragmented,cactus,0.429,0.968
1,E. coli fragmented,cactus (powyżej progu pokrycia),0.416,0.967
4,Klebsiella,cactus,0.229,0.957
5,Klebsiella,cactus (powyżej progu pokrycia),0.224,0.957
8,Pneumococcus,cactus,0.364,0.958
9,Pneumococcus,cactus (powyżej progu pokrycia),0.357,0.957


In [274]:
df_stats.to_csv("/home/pampuch/studia/magisterka/final_summaries/bedtools_core/stats_all.csv", index=False)

### Ponieważ dla mniektórych genomów uzyskane wartośći są dużo gorsze, należy sprawedzić czy to nie z powodu braku obecności scaffoldow w modelach
1. Sprawdzić, jaka jest część wspólna
2. Pocjonalnie, przygptwać analogiczne statystyki dla kowystępujących scaffoldów

In [275]:
all_summaries = all_sumamries

In [276]:
scaff_names = {dataset : {} for dataset in all_sumamries}
for dataset, models in all_sumamries.items():
    # truth = all_sumamries[dataset]["panaroo"]["all"]
    # print(len(truth))
    for model, core_status in models.items():
        # if model == "panaroo" or model == "source data":
        #     continue
        # inter_ann = truth.intersection(core_status["all"])
        # model_n = len(core_status["all"])
        # recall = len(inter_ann) / len(truth)
        # prec = len(inter_ann) / model_n
        # stats.append([dataset, model, recall, prec])
        if model in {"source data", "panaroo", "cactus"}:
            # print(list(core_status["all"])[:3])
            # print([(".").join(i.split(".")[:-1]) for i in core_status["all"]][:3])
            scaff_names[dataset][model] = set([(".").join(i.split(".")[:-1]) for i in core_status["all"]])
            

In [277]:
for dataset in scaff_names:
    # common_scaffs = 
    scaff_names[dataset]["Część wspólna Cactus i Panaroo"] = scaff_names[dataset]["panaroo"].intersection(scaff_names[dataset]["cactus"])

In [278]:
scaff_n = []
columns = ["Dataset","source data", "panaroo", "cactus", "Część wspólna Cactus i Panaroo"]
for dataset, models in scaff_names.items():
    # print(scaff_names[dataset])
    print(dataset)
    for model, names in models.items():
        print(model)
        print(len(list(names)))
    scaff_n.append([dataset] + [len(list(models[model])) for model in columns[1:]])
    print("-"*30)

sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented
panaroo
24150
cactus
14933
source data
26222
Część wspólna Cactus i Panaroo
14551
------------------------------
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated
panaroo
6812
cactus
5740
source data
8506
Część wspólna Cactus i Panaroo
5623
------------------------------
klebsiella_subset
panaroo
7048
cactus
3772
source data
8281
Część wspólna Cactus i Panaroo
3492
------------------------------
sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1
panaroo
6158
cactus
4900
source data
6647
Część wspólna Cactus i Panaroo
4796
------------------------------
GPSC_subset
panaroo
848
cactus
484
source data
1118
Część wspólna Cactus i Panaroo
438
------------------------------


In [279]:
scaff_n

[['sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_fragmented',
  26222,
  24150,
  14933,
  14551],
 ['sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1_contaminated', 8506, 6812, 5740, 5623],
 ['klebsiella_subset', 8281, 7048, 3772, 3492],
 ['sim_gr_1e-11_lr_1e-12_mu_1e-15_rep1', 6647, 6158, 4900, 4796],
 ['GPSC_subset', 1118, 848, 484, 438]]

In [280]:
df_scaff_no = pd.DataFrame(columns = columns, data = scaff_n)
df_scaff_no["Dataset"] = df_scaff_no["Dataset"].apply(datasets_mapping)
df_scaff_no = df_scaff_no.sort_values(by=["Dataset"])
df_scaff_no = df_scaff_no.rename(columns = {col : col.title() for col in df_scaff_no.columns[:-1]})

In [281]:
df_scaff_no

,Dataset,Source Data,Panaroo,Cactus,Część wspólna Cactus i Panaroo
3,E. coli,6647,6158,4900,4796
1,E. coli contaminated,8506,6812,5740,5623
0,E. coli fragmented,26222,24150,14933,14551
2,Klebsiella,8281,7048,3772,3492
4,Pneumococcus,1118,848,484,438


In [282]:
df_scaff_no.to_csv("/home/pampuch/studia/magisterka/final_summaries/bedtools_core/scaffs_no.csv", index=False)

### Tutaj faktycznie bardziej sensowne mogłoby być pokrycie tych scaffoldów. No może i jest, ale może być bardzo mały fragment. Dodaj może rozkład pokrycia

### Dostajesz 30 minut na spięcie informacji i scaffoldach. Potem przechodzisz do uzupełnień panaroo (45 min), potem przerwa i spisujesz wyniki